# Modify an NDEx Network using NiceCX

In this notebook, you will retrieve a public example network from NDEx, modify it, and then write it as a new, private network in your account. Optionally, you can modify and update a network that you own. 

import the ndex2 Python client

In [1]:
import ndex2
import itertools

Next, set a variable to hold the UUID of the network we want to modify

In [17]:
# fails
my_network_uuid = '601902a1-d632-11e7-adc1-0ac135e8bacf'

# works
#my_network_uuid = '9ed0cd55-9ac0-11e4-9499-000c29202374'

Set variables to hold your account and password 

In [21]:
my_server = "public.ndexbio.org"
my_account = "my_account"
my_password = "my_password"


try:
    my_ndex=ndex2.client.Ndex2("http://public.ndexbio.org", my_account, my_password)
    my_ndex.update_status()
    networks = my_ndex.status.get("networkCount")
    users = my_ndex.status.get("userCount")
    groups = my_ndex.status.get("groupCount")
    print("my_ndex client: %s networks, %s users, %s groups" % (networks, users, groups))
    response=my_ndex.get_network_as_cx_stream(my_network_uuid)
    print("Received %s characters of CX" % len(response.content))
except Exception as inst:
    print("Could not access account %s with password %s" % (my_account, my_password))
    print(inst.args)
    


my_ndex client: 17231 networks, 896 users, 90 groups
Received 11062103 characters of CX


Create a NiceCX network from the example network in the NDEx Tutorials account. If you would rather use a network in your own directory - perhaps made by cloning the example network - then provide your account and password as shown in the commented line.

In [22]:
print("downloading network and buiding NiceCX...")

    
my_network = ndex2.create_nice_cx_from_server(server='public.ndexbio.org', uuid=my_network_uuid)
#my_network = ndex2.create_nice_cx_from_server(server='public.ndexbio.org', uuid=my_network_uuid, account=my_account, password = my_password)
print("done")
print(my_network.get_summary())

downloading network and buiding NiceCX...


TypeError: 'NoneType' object is not iterable

The plan is to modify the network by merging two node attributes into one, where the two attributes were created by loading a table of edges with source and target node colums.

Steps:

* Iterate over all the nodes in the network and for each one we check the values of the two "source" attributes. 
* Add the new attribute based on the OR of the source attributes. 
* Finally, delete the two original source attributes


In [5]:
# (for clarity, this example code is rather verbose)

source_attribute1 = "TYPEA"
source_attribute2 = "TYPEB"
target_attribute = "TYPE"

for node_id, node in my_network.nodes.items():
    value1 = my_network.get_node_attribute(node, source_attribute1)
    value2 = my_network.get_node_attribute(node, source_attribute2)
    merged_value = value1 or value2
    if merged_value:
        my_network.set_node_attribute(node, target_attribute, merged_value)
        my_network.remove_node_attribute(node, source_attribute1)
        my_network.remove_node_attribute(node, source_attribute2)


Spot-check the network by printing out a sample of the nodes in the modified network

In [6]:
for id, node in itertools.islice(my_network.get_nodes(), 5):
    attribute_value = my_network.get_node_attribute(node, target_attribute)
    print("%s: %s = %s" % (node.get_name(), target_attribute, attribute_value))


PtdIns3P: TYPE = smallmolecule
prostaglandin D2: TYPE = smallmolecule
Prostacycline: TYPE = smallmolecule
INPP4B: TYPE = protein
Leukotriene A4: TYPE = smallmolecule


Upload the network to your NDEx account as a new network.

If you started by loading an one of your networks (such as a clone of the example network), then you can *update* it instead, overwriting its content. 

In the commented line of below, the optional parameter *update_uuid* specifies the update behavior.

In [7]:
upload_message = my_network.upload_to(my_server, my_account, my_password)
# update_message = my_network.update(my_server, my_account, my_password, update_uuid=my_network_uuid)
print(upload_message)

http://public.ndexbio.org/v2/network/c895c6c5-d45f-11e7-adc1-0ac135e8bacf
